In [ ]:
# !pip install keras-tuner

In [ ]:
# !pip install --upgrade tensorflow keras-tuner

In [ ]:
import os

import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers, models
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

import keras_tuner as kt
from keras import regularizers

import gc

# Importing and transforming dataset

In [ ]:
def label_counter(inpt, data_name):
    c = 0
    u = 0
    for i in inpt:
        if i == 1:
            c = c+1
        elif i == 0:
            u = u + 1
    print(f'{data_name} --> Traffic related: {c}, Traffic unrelated: {u}')

def import_dataset(split_seed):

    #First dataset
    # traffic_related = pl.read_csv('/content/drive/MyDrive/congestion_detector_datasets/model1_traffic_related_dataset.csv')

    #Second dtaset
    # traffic_unrelated = pl.read_csv('/content/drive/MyDrive/congestion_detector_datasets/model1_traffic_unrelated_dataset.csv')

    #Combining first and second dataset
#     df = pl.concat([
#         pl.read_csv('/kaggle/input/cardboard-congested-uncongested-dataset/congested_cardboard.csv'),
#         pl.read_csv('/kaggle/input/cardboard-congested-uncongested-dataset/uncongested_cardboard.csv')]
#         , how="vertical")
    df = pl.read_csv('/kaggle/input/cardboard-congestion-detector/congested_uncongested_cardboard.csv')

    y = df[:, 0] # Getting labels as series
    y = y.to_numpy()

    df = df[:, 1:]/255.0 # Normalizing pixels value to range of 0 to 1
    df = df.to_numpy().reshape(-1, 222, 296, 1) #reshaping to 296 by 222

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.1, random_state=split_seed) #42

    # Split train data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=split_seed)

    label_counter(y_train, 'y_train')
    label_counter(y_test, 'y_test')
    label_counter(y_val, 'y_val')

    return X_train, y_train, X_test, y_test, X_val, y_val

X_train, y_train, X_test, y_test, X_val, y_val = import_dataset(123)

# Exploring dataset

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define class labels
class_labels = {0: 'Uncongested', 1: 'Congested'}

# Map class labels for visualization
train_classes = np.array([class_labels[label] for label in y_train])
test_classes = np.array([class_labels[label] for label in y_test])
val_classes = np.array([class_labels[label] for label in y_val])

# Calculate class frequencies
train_class_counts = np.unique(train_classes, return_counts=True)
test_class_counts = np.unique(test_classes, return_counts=True)
val_class_counts = np.unique(val_classes, return_counts=True)

# Create bar chart data
data = [
    go.Bar(x=train_class_counts[0], y=train_class_counts[1], name='Train'),
    go.Bar(x=test_class_counts[0], y=test_class_counts[1], name='Test'),
    go.Bar(x=val_class_counts[0], y=val_class_counts[1], name='Validation')
]

# Set layout
layout = go.Layout(
    title='Occurrences of Binary Classes',
    xaxis=dict(title='Class'),
    yaxis=dict(title='Count'),
    barmode='group'
)

# Create figure
fig = go.Figure(data=data, layout=layout)

# Show the bar chart
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define class labels
class_labels = {0: 'Uncongested', 1: 'Congested'}

# Calculate total class counts
class_counts = {
    class_labels[0]: np.sum([np.sum(y_train == 0), np.sum(y_test == 0), np.sum(y_val == 0)]),
    class_labels[1]: np.sum([np.sum(y_train == 1), np.sum(y_test == 1), np.sum(y_val == 1)])
}

# Create bar chart data
data = [
    go.Bar(x=list(class_counts.keys()), y=list(class_counts.values()))
]

# Set layout
layout = go.Layout(
    title='Total Occurrences of Binary Classes',
    xaxis=dict(title='Class'),
    yaxis=dict(title='Count')
)

# Create figure
fig = go.Figure(data=data, layout=layout)

# Show the bar chart
fig.show()

In [ ]:
def image_visualizer(X, y, row_number):
    X = X[row_number]
    y = y[row_number]

    plt.imshow(X, cmap='gray')
    lbl = None
    if y == 0:
        lbl = 'Uncongested'
    elif y == 1:
        lbl = 'congested'
    plt.title(lbl)
    plt.show()

image_visualizer(X_train, y_train, 5)
image_visualizer(X_train, y_train, 1345)
image_visualizer(X_train, y_train, 512)
image_visualizer(X_train, y_train, 688)

# Further data transformations

In [ ]:
import numpy as np

def transformer_func(X):
  datagen = keras.preprocessing.image.ImageDataGenerator(
      rotation_range=15,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest'
  )

  # Create empty list for augmented data
  X_augmented = []

  transformed_image_counter = 0

  # Define probability threshold
  probability_threshold = 0.5

  # Apply data augmentation to each image individually
  for i in range(len(X)):
      # Generate a random number between 0 and 1
      random_probability = np.random.uniform(0, 1)

      # Apply transformations only if random_probability is above the threshold
      if random_probability > probability_threshold:
          augmented_image = datagen.apply_transform(X[i], datagen.get_random_transform(X[i].shape))
          X_augmented.append(augmented_image)
          transformed_image_counter += 1
      else:
          X_augmented.append(X_train[i])

  # Convert augmented data to array
  X_train_augmented = np.array(X_augmented)

  return np.array(X_augmented), transformed_image_counter

X_train, transformed_image_counter = transformer_func(X_train)

print(f'{transformed_image_counter} images transformed.')

In [ ]:
X_train.shape

In [ ]:
# from keras.utils import to_categorical

# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)
# y_val = to_categorical(y_val)

In [ ]:
# image_visualizer(X_train, y_train, 5)
# image_visualizer(X_train, y_train, 3005)
# image_visualizer(X_train, y_train, 662)
# image_visualizer(X_train, y_train, 2235)

# Model hyperparameter tuning

In [ ]:
# Trial 21 Complete [00h 00m 54s]
# val_accuracy: 0.9497607946395874

# Best val_accuracy So Far: 0.9497607946395874
# Total elapsed time: 00h 10m 29s

# Search: Running Trial #22

# Value             |Best Value So Far |Hyperparameter
# 75                |35                |filter_1
# 5                 |5                 |filter_size_1
# 2                 |3                 |kernel_size_1
# 30                |40                |filter_2
# 2                 |3                 |filter_size_2
# 1                 |2                 |kernel_size_2
# 30                |25                |filter_3
# 2                 |2                 |filter_size_3
# 1                 |3                 |kernel_size_3
# 35                |35                |filter_4
# 2                 |2                 |filter_size_4
# 3                 |3                 |kernel_size_4
# 50                |30                |filter_5
# 2                 |2                 |filter_size_5
# 3                 |1                 |kernel_size_5
# 70                |60                |filter_6
# 3                 |3                 |filter_size_6
# 1                 |2                 |kernel_size_6
# 25                |65                |units_1
# 0.2               |0.1               |dropout_rate
# 20                |15                |units_2
# 0.0003            |0.0003            |learning_rate
# 10                |10                |tuner/epochs
# 4                 |4                 |tuner/initial_epoch
# 1                 |1                 |tuner/bracket
# 1                 |1                 |tuner/round
# 0018              |0017              |tuner/trial_id


In [ ]:
#### import tensorflow as tf
import keras_tuner as kt

def model_builder(hp):
    model = keras.Sequential()
    
    model.add(keras.layers.Conv2D(hp.Int('filter_1', min_value=16, max_value=126, step=16), hp.Choice('filter_size_1', values=[2,3]), activation='relu', input_shape=(222, 296, 1)))
    model.add(keras.layers.Conv2D(hp.Int('filter_2', min_value=16, max_value=126, step=16), hp.Choice('filter_size_2', values=[2,3]), strides=hp.Choice('stride_size1_', values=[2,3]), padding='same', activation='relu'))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_1', values=[2,3]), strides=(2, 2)))
    
    model.add(keras.layers.Conv2D(hp.Int('filter_3', min_value=8, max_value=64, step=8), hp.Choice('filter_size_3', values=[2,3]), strides=hp.Choice('stride_size_2', values=[1,2]), kernel_regularizer=regularizers.l2(hp.Float('l2_rate_1', min_value=0.0, max_value=0.05, step=0.01)), padding='valid', activation='relu'))
    model.add(keras.layers.Conv2D(hp.Int('filter_4', min_value=8, max_value=64, step=8), hp.Choice('filter_size_4', values=[2,3]), strides=hp.Choice('stride_size_3', values=[1,2]), padding='valid', activation='relu'))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_2', values=[2,3]), strides=(2,2)))
     
    model.add(keras.layers.Conv2D(hp.Int('filter_5', min_value=4, max_value=32, step=4), hp.Choice('filter_size_5', values=[1,2]), strides=hp.Choice('stride_size_4', values=[1,2]), padding='valid', activation='relu'))
    model.add(keras.layers.Conv2D(hp.Int('filter_6', min_value=4, max_value=32, step=4), hp.Choice('filter_size_6', values=[1,2]), strides=hp.Choice('stride_size_5', values=[1,2]), padding='valid', activation='relu'))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_3', values=[1,2]), strides=(1,1)))
    
    model.add(keras.layers.Conv2D(hp.Int('filter_7', min_value=4, max_value=126, step=16), hp.Choice('filter_size_7', values=[1,2]), strides=hp.Choice('stride_size_6', values=[1,2]), kernel_regularizer=regularizers.l2(hp.Float('l2_rate_2', min_value=0.0, max_value=0.05, step=0.01)), padding='valid', activation='relu'))
    model.add(keras.layers.Conv2D(hp.Int('filter_8', min_value=4, max_value=126, step=16), hp.Choice('filter_size_8', values=[1,2]), strides=hp.Choice('stride_size_7', values=[1,2]), padding='same', activation='relu'))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_4', values=[1,2]), strides=(1,1)))
              
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=hp.Int('units_1', min_value=4, max_value=64, step=8), activation='relu'))
    
    hp_dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    model.add(keras.layers.Dropout(hp_dropout_rate))
    
    model.add(keras.layers.Dense(hp.Int('units_2', min_value=4, max_value=126, step=8), activation='relu'))
    
    model.add(keras.layers.Dense(2, activation='softmax'))
#     model.add(keras.layers.Dense(1, activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0003, 0.0001])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    return model

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='/kaggle/working/',
                     project_name='congested_vs_uncongested_traffic_detector9')
              
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), callbacks=[stop_early])

In [ ]:
# import tensorflow as tf
# import keras_tuner as kt

# def model_builder(hp):
#     model = keras.Sequential()
    
#     model.add(keras.layers.Conv2D(
#         hp.Int('filter_1', min_value=5, max_value=200, step=10), 
#         hp.Choice('filter_size_1', values=[3, 4, 5]), 
#         activation='relu', 
#         input_shape=(222, 296, 1)))
#     model.add(keras.layers.Conv2D(
#         hp.Int('filter_2', min_value=5, max_value=150, step=5), 
#         hp.Choice('filter_size_2', values=[3, 4]), 
# #         padding = (2,2),
# #         kernel_regularizer=regularizers.l2(hp.Float('l2_rate_1', min_value=0.0, max_value=0.05, step=0.01))
#     ))
#     model.add(keras.layers.MaxPooling2D(
#         hp.Choice('kernel_size_1', values=[3, 4]), 
#         strides=(2, 2)))
    
#     model.add(keras.layers.Conv2D(
#         hp.Int('filter_3', min_value=5, max_value=64, step=10), 
#         hp.Choice('filter_size_3', values=[3, 4]),
#         activation='relu'))
#     model.add(keras.layers.Conv2D(
#         hp.Int('filter_4', min_value=5, max_value=64, step=10), 
#         hp.Choice('filter_size_4', values=[3, 4]), 
# #         padding = (2,2),
# #         kernel_regularizer=regularizers.l2(hp.Float('l2_rate_2', min_value=0.0, max_value=0.2, step=0.06))
#     ))
#     model.add(keras.layers.MaxPooling2D(
#         hp.Choice('kernel_size_2', values=[3, 4]), 
#         strides=(3, 3)))
              
# #     model.add(keras.layers.Conv2D(
# #         hp.Int('filter_5', min_value=5, max_value=65, step=5), 
# #         hp.Choice('filter_size_5', values=[3, 4]), 
# #         activation='relu'))
# #     model.add(keras.layers.Conv2D(
# #         hp.Int('filter_6', min_value=5, max_value=65, step=5), 
# #         hp.Choice('filter_size_6', values=[3, 4]))) 
# # #         kernel_regularizer=regularizers.l2(hp.Float('l2_rate_3', min_value=0.0, max_value=0.05, step=0.01))))
# #     model.add(keras.layers.MaxPooling2D(
# #         hp.Choice('kernel_size_3', values=[3, 4]), 
# #         strides=(3, 3)))
              
#     model.add(keras.layers.Conv2D(
#         hp.Int('filter_7', min_value=5, max_value=100, step=5), 
#         hp.Choice('filter_size_7', values=[2, 3]), 
#         activation='relu'))
#     model.add(keras.layers.Conv2D(
#         hp.Int('filter_8', min_value=5, max_value=100, step=5), 
#         hp.Choice('filter_size_8', values=[2, 3]), 
# #         padding = (2,2),
# #         kernel_regularizer=regularizers.l2(hp.Float('l2_rate_4', min_value=0.0, max_value=0.05, step=0.01))
#     ))
#     model.add(keras.layers.MaxPooling2D(
#         hp.Choice('kernel_size_4', values=[2, 3]), 
#         strides=(2, 2)))
              
#     model.add(keras.layers.Flatten())
#     model.add(keras.layers.Dense(units=hp.Int('units_1', min_value=10, max_value=200, step=10), activation='relu'))
    
#     hp_dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
#     model.add(keras.layers.Dropout(hp_dropout_rate))
    
#     model.add(keras.layers.Dense(hp.Int('units_2', min_value=5, max_value=100, step=10), activation='relu'))
    
# #     model.add(keras.layers.Dense(2, activation='softmax'))
#     model.add(keras.layers.Dense(1, activation='sigmoid'))

#     hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0003, 0.0001])

#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
#                   loss=keras.losses.SparseCategoricalCrossentropy(),
#                   metrics=['accuracy'])

#     return model

# tuner = kt.Hyperband(model_builder,
#                      objective='val_accuracy',
#                      max_epochs=10,
#                      factor=3,
#                      directory='/kaggle/working/',
#                      project_name='congested_vs_uncongested_traffic_detector')
              
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# tuner.search(X_train, y_train, epochs=15, validation_data=(X_val, y_val), callbacks=[stop_early])

# Building Model

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

best_hps.values

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    batch_size=64,
                    epochs=50)

In [ ]:
# model.save(f'/kaggle/working/{accuracy:.3f}accuracy_{loss:.3f}loss_cardboard_model')

model.save(f'/kaggle/working/model1')

loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

# Re-instantiate the hypermodel with the optimal number of epochs

In [ ]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train,
                    epochs=best_epoch,
                    batch_size=64,
                    validation_data=(X_val, y_val))

In [ ]:
loss1, accuracy1 = hypermodel.evaluate(X_test, y_test)
print('Loss:', loss1)
print('Accuracy:', accuracy1)

In [ ]:
# cnn.save(f'{accuracy:.3f}accuracy_{loss:.3f}loss_cardboard_model')
hypermodel.save(f'/kaggle/working/model2')